In [13]:
import numpy as np #import numpy library
import pandas as pd #import pandas library
import matplotlib.pyplot as plt #import matplot library
%matplotlib inline

import glob
import os

import nltk.classify.util
from astropy.table import Table, Column

import string

from sklearn import svm
from sklearn import metrics


In [14]:
 def loadMetricData(checkoutCenter, timeFrame):
    path =  checkoutCenter + "/" + timeFrame + "/"
#     print(path)
    all_files = glob.glob(os.path.join(path, "*.csv")) 
#     print(all_files)
    data = pd.DataFrame()
    list_ = []
    for file_ in all_files:
        data = pd.read_csv(file_,index_col=None, header=0)
        list_.append(data)
        data = pd.concat(list_)
    return data

In [15]:
metrics = loadMetricData('ERC', "06222016_06232016")

metrics.tail(5)

,Allocation,Latest Operator,Patron,Patron Barcode,Patron Department,Patron Email,Renewal Count,Account Code,Actual Return Time,Aggregate Value,...,Return Time,Returning Operator,Route Summary,Set People,Set/Strike Info,Show,State,Strike People,Use Charge Duration,Summary
115,CK-171588,Colin Todd,Barbara Compagnoni,NaN,"PSD, School of Art, Media and Technology",compagnb@newschool.edu,0,NaN,NaN,"$5,900.00",...,"Jun 22, 2016 2:37 PM",NaN,NaN,NaN,NaN,F,CHECKOUT,NaN,F,Checked Out: ADAPTERS / MOUNTS / STANDS - LIGH...
116,CK-171589,Amanda Alfieri,Barbara Compagnoni,NaN,"PSD, School of Art, Media and Technology",compagnb@newschool.edu,0,NaN,"Jun 21, 2016 5:46 PM","$1,000.00",...,"Jun 22, 2016 3:11 PM",Colin Todd,NaN,NaN,NaN,F,CHECKOUT-COMPLETED,NaN,F,Returned: CANON XA10 - CANON XA10 HD W/ MICROP...
117,CK-171599,Colin Todd,Hashem Eaddy,NaN,Photography,eaddyh@newschool.edu,1,NaN,NaN,"$2,496.00",...,"Jun 27, 2016 9:00 AM",NaN,NaN,NaN,NaN,F,CHECKOUT,NaN,F,Checked Out: A7 - B21SY - A7 28-70MM - B21SY |...
118,CK-171600,Colin Todd,Hashem Eaddy,NaN,Photography,eaddyh@newschool.edu,0,NaN,NaN,$0.00,...,"Aug 1, 2016 9:00 AM",NaN,NaN,NaN,NaN,F,CHECKOUT,NaN,F,Checked Out: AMT WACOM TABLETS - WACOM INTROUS...
119,CK-171622,Amanda Alfieri,Sol Hashemi,NaN,"PSD, School of Art, Media and Technology",sol@newschool.edu,0,NaN,NaN,$400.00,...,"Jun 23, 2016 5:27 PM",NaN,NaN,NaN,NaN,F,CHECKOUT,NaN,F,Checked Out: PROFOTO AIR REMOTE - PROFOTO AIR ...


In [36]:
metrics.dtypes

Allocation                        object
Latest Operator                   object
Patron                            object
Patron Barcode                   float64
Patron Department                 object
Patron Email                      object
Renewal Count                      int64
Account Code                     float64
Actual Return Time        datetime64[ns]
Aggregate Value                   object
Allocation Notes                  object
Auto-Circulate                    object
Checkout Center                   object
Class Or Event Title             float64
Contact Info                     float64
Contact Person                   float64
Charge Duration                  float64
Creation Time                     object
Credit Card Type                 float64
Default Patron Class              object
Deliver To Location              float64
Deliver To Name                  float64
Deliver To String                float64
Delivery Type                     object
Effective Patron

In [17]:
# clean metric data
metrics['Actual Return Time'] = pd.to_datetime(metrics['Actual Return Time'])
metrics['Return Time'] = pd.to_datetime(metrics['Return Time'])
metrics['Pickup Time'] = pd.to_datetime(metrics['Pickup Time'])

# metrics.dtypes
# metrics.head(5)

,Allocation,Latest Operator,Patron,Patron Barcode,Patron Department,Patron Email,Renewal Count,Account Code,Actual Return Time,Aggregate Value,...,Return Time,Returning Operator,Route Summary,Set People,Set/Strike Info,Show,State,Strike People,Use Charge Duration,Summary
0,CK-171407,Amanda Alfieri,James L. Ramer,2.120701e+13,NaN,ramerj@newschool.edu,0,NaN,2016-06-14 17:26:00,$0.00,...,2016-06-15 15:45:00,Amanda Alfieri,NaN,NaN,NaN,F,CHECKOUT-COMPLETED,NaN,F,Returned: MINI DISPLAY PORT TO HDMI - MINI DIS...
1,CK-171248,Administrator,Miranda Lichtenstein,2.120701e+13,NaN,lichtenm@newschool.edu,0,NaN,NaT,"$5,999.00",...,2016-06-16 15:30:00,NaN,NaN,NaN,NaN,F,RESERVATION-CANCELLED,NaN,F,Reservation Cancelled: LARGE FORMAT - COURTNEY...
2,CK-171249,Administrator,Miranda Lichtenstein,2.120701e+13,NaN,lichtenm@newschool.edu,0,NaN,NaT,"$5,999.00",...,2016-06-11 13:30:00,NaN,NaN,NaN,NaN,F,RESERVATION-CANCELLED,NaN,F,Reservation Cancelled: LARGE FORMAT - COURTNEY...
3,CK-171584,Amanda Alfieri,Miranda Lichtenstein,2.120701e+13,NaN,lichtenm@newschool.edu,0,NaN,NaT,"$5,999.00",...,2016-06-23 16:30:00,NaN,NaN,NaN,NaN,F,RESERVATION,NaN,F,Reserved: LARGE FORMAT - COURTNEY LOVE 9900 - ...
4,CK-171345,Amanda Alfieri,Sebastian Matias Perinotti,2.120710e+13,MPH,peris942@newschool.edu,0,NaN,2016-06-14 12:23:00,"$1,827.00",...,2016-06-14 12:30:00,Amanda Alfieri,NaN,NaN,NaN,F,CHECKOUT-COMPLETED,NaN,F,Returned: FUJI DIGITAL - X-PRO1 18-55MM B01FD ...


In [47]:
# Cameras

# TEST
# metrics['Jim'] = metrics['Patron'].str.contains('James L. Ramer')

# Sony
metrics['Sony a99'] = metrics['Summary'].str.contains('A99')
metrics['Sony a99 count'] = metrics['Summary'].str.count('A99')
metrics['Sony a77'] = metrics['Summary'].str.contains('A77')
metrics['Sony a77 count'] = metrics['Summary'].str.count('A77')
metrics['Sony a65'] = metrics['Summary'].str.contains('A65')
metrics['Sony a65 count'] = metrics['Summary'].str.count('A65')
metrics['Sony NEX-7'] = metrics['Summary'].str.contains('NEX-7')
metrics['Sony NEX-7 count'] = metrics['Summary'].str.count('NEX-7')
metrics['Sony a7'] = metrics['Summary'].str.contains('A7')
metrics['Sony a7 count'] = metrics['Summary'].str.count('A7')
metrics['Sony RX1'] = metrics['Summary'].str.contains('RX1')
metrics['Sony RX1 count'] = metrics['Summary'].str.count('RX1')

# Canon
metrics['Canon 1DS'] = metrics['Summary'].str.contains('1DS')
metrics['Canon 1DS count'] = metrics['Summary'].str.count('1DS')
metrics['Canon 1DSMII'] = metrics['Summary'].str.contains('1DS MARK II')
metrics['Canon 1DSMII count'] = metrics['Summary'].str.count('1DS MARK II')
metrics['Canon 1DSMIII'] = metrics['Summary'].str.contains('1DS MARK III')
metrics['Canon 1DSMIII count'] = metrics['Summary'].str.count('1DS MARK III')
metrics['Canon 1DSMIV'] = metrics['Summary'].str.contains('1DS MARK IV')
metrics['Canon 1DSMIV count'] = metrics['Summary'].str.count('1DS MARK IV')
metrics['Canon 20D'] = metrics['Summary'].str.contains('20D') 
metrics['Canon 20D count'] = metrics['Summary'].str.count('20D')
metrics['Canon 40D'] = metrics['Summary'].str.contains('40D')
metrics['Canon 40D count'] = metrics['Summary'].str.count('40D')
metrics['Canon 50D'] = metrics['Summary'].str.contains('50D')
metrics['Canon 50D count'] = metrics['Summary'].str.count('50D')
metrics['Canon 5D'] = metrics['Summary'].str.contains('5D')
metrics['Canon 5D count'] = metrics['Summary'].str.count('5D')
metrics['Canon 5DMII'] = metrics['Summary'].str.contains('5D MARK II')
metrics['Canon 5DMII count'] = metrics['Summary'].str.count('5D MARK II')
metrics['Canon 5DMIII'] = metrics['Summary'].str.contains('5D MARK III')
metrics['Canon 5DMIII count'] = metrics['Summary'].str.count('5D MARK III')
metrics['Canon 5DSR'] = metrics['Summary'].str.contains('5D SR')
metrics['Canon 5DSR count'] = metrics['Summary'].str.count('5D SR')
metrics['Canon 7D'] = metrics['Summary'].str.contains('7D')
metrics['Canon 7D count'] = metrics['Summary'].str.count('7D')

# Nikon
metrics['Nikon D40'] = metrics['Summary'].str.contains('D40')
metrics['Nikon D40 count'] = metrics['Summary'].str.count('D40')
metrics['Nikon D70'] = metrics['Summary'].str.contains('D70')
metrics['Nikon D40 count'] = metrics['Summary'].str.count('D70')
metrics['Nikon D3'] = metrics['Summary'].str.contains('D3 ')
metrics['Nikon D40 count'] = metrics['Summary'].str.count('D3 ')
metrics['Nikon D300'] = metrics['Summary'].str.contains('D300S')
metrics['Nikon D40 count'] = metrics['Summary'].str.count('D300S')
metrics['Nikon D700'] = metrics['Summary'].str.contains('D700')
metrics['Nikon D40 count'] = metrics['Summary'].str.count('D700')
metrics['Nikon D800'] = metrics['Summary'].str.contains('D800')
metrics['Nikon D40 count'] = metrics['Summary'].str.count('D800')
metrics['Nikon F50MM'] = metrics['Summary'].str.contains('NIKON F')
metrics['Nikon D40 count'] = metrics['Summary'].str.count('NIKON F')

# Date and Time

# Check out
metrics['ckout weekday'] = metrics['Pickup Time'].dt.weekday
metrics['ckout month'] = metrics['Pickup Time'].dt.month
metrics['ckout hour'] = metrics['Pickup Time'].dt.hour
metrics['ckout min'] = metrics['Pickup Time'].dt.minute

# Check in
metrics['ckin weekday'] = metrics['Actual Return Time'].dt.weekday
metrics['ckin month'] = metrics['Actual Return Time'].dt.month
metrics['ckin hour'] = metrics['Actual Return Time'].dt.hour
metrics['ckin min'] = metrics['Actual Return Time'].dt.minute

# Late
metrics['late'] = 
# metrics['weekday'] = totalData['timestamp'].dt.weekday
# totalData['month'] = totalData['timestamp'].dt.month
# totalData['hour'] = totalData['timestamp'].dt.hour
# totalData['min'] = totalData['timestamp'].dt.minute



In [48]:
metrics.head(5)


,Allocation,Latest Operator,Patron,Patron Barcode,Patron Department,Patron Email,Renewal Count,Account Code,Actual Return Time,Aggregate Value,...,Nikon D800,Nikon F50MM,ckout weekday,ckout hour,ckout min,ckout month,ckin weekday,ckin month,ckin hour,ckin min
0,CK-171407,Amanda Alfieri,James L. Ramer,2.120701e+13,NaN,ramerj@newschool.edu,0,NaN,2016-06-14 17:26:00,$0.00,...,False,False,1,15,46,6,1.0,6.0,17.0,26.0
1,CK-171248,Administrator,Miranda Lichtenstein,2.120701e+13,NaN,lichtenm@newschool.edu,0,NaN,NaT,"$5,999.00",...,False,False,3,12,30,6,NaN,NaN,NaN,NaN
2,CK-171249,Administrator,Miranda Lichtenstein,2.120701e+13,NaN,lichtenm@newschool.edu,0,NaN,NaT,"$5,999.00",...,False,False,5,10,30,6,NaN,NaN,NaN,NaN
3,CK-171584,Amanda Alfieri,Miranda Lichtenstein,2.120701e+13,NaN,lichtenm@newschool.edu,0,NaN,NaT,"$5,999.00",...,False,False,3,13,30,6,NaN,NaN,NaN,NaN
4,CK-171345,Amanda Alfieri,Sebastian Matias Perinotti,2.120710e+13,MPH,peris942@newschool.edu,0,NaN,2016-06-14 12:23:00,"$1,827.00",...,False,False,5,14,5,6,1.0,6.0,12.0,23.0
